# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-54ebe87aca-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("American Architects") 

Consider the following exploratory information need:

> You want to study the most prolific american architects

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q5604`    | Frank Lloyd Wright     | node      |
| `wd:Q30`      | U.S.A.        | node |
| `wd:Q42973`   | architect     | node |
| `wd:Q2081276` | Pettit Memorial Chapel        | node |
| `wd:Q2977`    | Cathedral     | node |



Also consider

```
?p wdt:P27 wd:Q30  . 
?p wdt:P106 wd:Q42973  . 
```

is the BGP to retrieve all **american architects**

## Workload Goals

1. Identify the BGP to obtain the notable works of an architect and their location

2. Identify the BGP that connect a building with its architect, type, and architectural style

3. Which architect is the most prolific in the U.S. and outside?

4. Analyze the location of the buildings designed by the american architects
 
   4.1 Which styles exists in the U.S.? Which type of building?
   
   4.2 In which U.S. states there are more notable buildings?
   
   4.3 Which U.S. state contains the largest number of building designed by U.S. architects


In [3]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P27 wd:Q30  . 
?p wdt:P106 wd:Q42973  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '3354')]


1

## Architects Notable Works
This section contains the queries used to answer the first question: _Identify the BGP to obtain the notable works of an architect and their location._

In order to answer to the question above firstly I need to find out how architects and buildings are linked together. I have been given an architect, Frank Lloyd Wright (wd:Q5604), and a building, Petit Memorial Chapel (wd:Q2081276), therefore I can see which properties are connected to these nodes. Firstly I count the properties where Frank Lloud Wright is the subject.

In [4]:
queryString = """
SELECT COUNT(?p) AS ?wrightProps
WHERE { 

wd:Q5604 ?p ?o.

} 
"""

print("Results")
run_query(queryString)

Results
[('wrightProps', '251')]


1

Since they are quite a lot I return them based on the number of objects connected to Wright by means of the same property. Since I am looking for connection between architects and building I can assume architects are connected to multiple buildings.

In [5]:
queryString = """
SELECT ?p ?pname COUNT(?o) AS ?objs
WHERE { 

wd:Q5604 ?p ?o.
?p <http://schema.org/name> ?pname.

} 
GROUP BY ?p ?pname
ORDER BY DESC(?objs)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('pname', 'has works in the collection'), ('objs', '9')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('objs', '7')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('objs', '6')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pname', 'child'), ('objs', '4')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('objs', '4')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'spouse'), ('objs', '3')]
[('p', 'http://www.wikidata.org/prop/direct/P6394'), ('pname', 'ELNET ID'), ('objs', '2')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('objs', '2')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('objs', '2')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image'), ('objs', '2')]
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('p

25

I discovered the property "notable work" (wdt:P800), therefore I return all triples involing it.

In [6]:
queryString = """
SELECT DISTINCT ?w ?work ?i ?instanceOf
WHERE { 

wd:Q5604 wdt:P800 ?w.
?w wdt:P31 ?i.
?w <http://schema.org/name> ?work.
?i <http://schema.org/name> ?instanceOf.

} 
"""

print("Results")
run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q4647666'), ('work', 'A. D. German Warehouse'), ('i', 'http://www.wikidata.org/entity/Q181623'), ('instanceOf', 'warehouse')]
[('w', 'http://www.wikidata.org/entity/Q1335804'), ('work', 'Emil Bach House'), ('i', 'http://www.wikidata.org/entity/Q1307276'), ('instanceOf', 'single-family detached home')]
[('w', 'http://www.wikidata.org/entity/Q463179'), ('work', 'Fallingwater'), ('i', 'http://www.wikidata.org/entity/Q1307276'), ('instanceOf', 'single-family detached home')]
[('w', 'http://www.wikidata.org/entity/Q3586059'), ('work', 'Annunciation Greek Orthodox Church'), ('i', 'http://www.wikidata.org/entity/Q16970'), ('instanceOf', 'church building')]
[('w', 'http://www.wikidata.org/entity/Q201469'), ('work', 'Solomon R. Guggenheim Museum'), ('i', 'http://www.wikidata.org/entity/Q207694'), ('instanceOf', 'art museum')]
[('w', 'http://www.wikidata.org/entity/Q201469'), ('work', 'Solomon R. Guggenheim Museum'), ('i', 'http://www.wikidata.org/e

9

I can assess this property retrieves an architect notable work. I also return the property "instance of" (wdt:P31) for each object in order to see what kind of objects the property returns and it looks like the type of building can be inferred by that property.

The second part of the requested BGP involves the location of the buildings therefore I need to retireve such information. Firsly, I count the properties Pettit Memorial Chapel (wd:Q2081276) is subject to.

In [7]:
queryString = """
SELECT COUNT(?p) AS ?petchapProps
WHERE { 

wd:Q2081276 ?p ?o.

} 
"""

print("Results")
run_query(queryString)

Results
[('petchapProps', '18')]


1

Since they are not so many I can return them all.

In [8]:
queryString = """
SELECT ?p ?pname COUNT(?o) AS ?objs
WHERE { 

wd:Q2081276 ?p ?o.
?p <http://schema.org/name> ?pname.

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('pname', 'Commons gallery'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pname', 'architectural style'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P649'), ('pname', 'NRHP reference number'), ('objs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity'), ('objs', '1')]
[

14

I discovered the building is connected to the property "country" (wdt:P17) that it is likely to contain the country where the building is located. I also discovered the property "architectural style" (wdt:P149) that could be useful for the following questions. In the end I discovered that each building is also connect by the property "architect" (wdt:P84) therefore I could see if by selecting all building linked to Wright by this property I get different results than before. Before checking that however I return the triple with the property "located in the administrative territorial entity" (wdt:P131) to see if this gives me some more accurate information about the location of a building.

In [9]:
queryString = """
SELECT ?o ?oname ?i ?instanceOf
WHERE { 

wd:Q2081276 wdt:P131 ?o.
?o wdt:P31 ?i.
?o <http://schema.org/name> ?oname.
?i <http://schema.org/name> ?instanceOf.

} 
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oname', 'Belvidere'), ('i', 'http://www.wikidata.org/entity/Q1093829'), ('instanceOf', 'city of the United States')]
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oname', 'Belvidere'), ('i', 'http://www.wikidata.org/entity/Q62049'), ('instanceOf', 'county seat')]


2

The above-mentioned property gives some more detailed information about the location of the buildings. I now check the properties where Pettit Chapel Memorial is the object in order to see if I find any other interesting properties.

In [10]:
queryString = """
SELECT ?p ?pname COUNT(?s) AS ?subjs
WHERE { 

?s ?p wd:Q2081276.
?p <http://schema.org/name> ?pname.

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic"), ('subjs', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('subjs', '1')]


2

I did not find any other useful property therefore I will only check the country where a building is located to.

I now check if Wright (wd:Q5604) is connected to anything by means of the property "architect" (wdt:P84).

In [11]:
queryString = """
SELECT COUNT(?s) AS ?subjs
WHERE { 

?s wdt:P84 wd:Q5604.
?s <http://schema.org/name> ?sname.

} 
"""

print("Results")
run_query(queryString)

Results
[('subjs', '244')]


1

I assess the property "architect" (wdt:P84) return all the works done by one architect therefore I will use such property when returning the most prolific architects. In this section however I am asked to retrieve the notable works therefore I will use the property "notable works" (wdt:P800).

To sum up what I discovered so far, to retrieve an architect's (?architect) notable work I use the triple: 
```
?architect wdt:P800 ?work.
```
To retrieve the country where a building (?building) is located I use the triple:
```
?building wdt:P17 ?country.
```
While to retrieve information about the city/county where it is located I use:
```
?building wdt:P131 ?location.
```

Now I can count the number of notable works each american architect have and return the top-25 american architects with more notable works.

In [12]:
queryString = """
SELECT ?a ?architect COUNT(DISTINCT ?work) AS ?notableWorks
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973;
    wdt:P800 ?work.
    
?a <http://schema.org/name> ?architect.

} 
GROUP BY ?a ?architect
ORDER BY DESC(?notableWorks)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q2926468'), ('architect', 'Bruce Price'), ('notableWorks', '17')]
[('a', 'http://www.wikidata.org/entity/Q5171172'), ('architect', 'Cornelia Oberlander'), ('notableWorks', '13')]
[('a', 'http://www.wikidata.org/entity/Q15996171'), ('architect', 'Egerton Swartwout'), ('notableWorks', '11')]
[('a', 'http://www.wikidata.org/entity/Q369263'), ('architect', 'John Trumbull'), ('notableWorks', '10')]
[('a', 'http://www.wikidata.org/entity/Q180374'), ('architect', 'Frank Gehry'), ('notableWorks', '9')]
[('a', 'http://www.wikidata.org/entity/Q102289'), ('architect', 'Buckminster Fuller'), ('notableWorks', '8')]
[('a', 'http://www.wikidata.org/entity/Q62918'), ('architect', 'Fazlur Khan'), ('notableWorks', '7')]
[('a', 'http://www.wikidata.org/entity/Q892140'), ('architect', 'Leopold Eidlitz'), ('notableWorks', '7')]
[('a', 'http://www.wikidata.org/entity/Q6180936'), ('architect', "Jeremiah O'Rourke"), ('notableWorks', '7')]
[('a', 'http://www.wikid

25

I can also count in how many different countries the notable works of an architects are located and return the top-15 american architects that worked in more different countries.

In [13]:
queryString = """
SELECT ?a ?architect COUNT(DISTINCT ?country) AS ?notableWCountries
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973;
    wdt:P800 ?work.
    
?work wdt:P17 ?country.
    
?a <http://schema.org/name> ?architect.

} 
GROUP BY ?a ?architect
ORDER BY DESC(?notableWCountries)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q180374'), ('architect', 'Frank Gehry'), ('notableWCountries', '5')]
[('a', 'http://www.wikidata.org/entity/Q46868'), ('architect', 'I. M. Pei'), ('notableWCountries', '4')]
[('a', 'http://www.wikidata.org/entity/Q309483'), ('architect', 'César Pelli'), ('notableWCountries', '4')]
[('a', 'http://www.wikidata.org/entity/Q374276'), ('architect', 'Adrian Smith'), ('notableWCountries', '4')]
[('a', 'http://www.wikidata.org/entity/Q41508'), ('architect', 'Ludwig Mies van der Rohe'), ('notableWCountries', '3')]
[('a', 'http://www.wikidata.org/entity/Q154538'), ('architect', 'Daniel Libeskind'), ('notableWCountries', '3')]
[('a', 'http://www.wikidata.org/entity/Q102289'), ('architect', 'Buckminster Fuller'), ('notableWCountries', '3')]
[('a', 'http://www.wikidata.org/entity/Q313828'), ('architect', 'Ivan Meštrović'), ('notableWCountries', '3')]
[('a', 'http://www.wikidata.org/entity/Q3756505'), ('architect', 'Diana Balmori Ling'), ('notableWCount

15

In the end, I return the average number of notable works done by architects of the same nationality.

In [14]:
queryString = """
SELECT ?nationality AVG(?works) AS ?avgWorks
WHERE { 

?a wdt:P27 ?c.
    
?c <http://schema.org/name> ?nationality.

{
    SELECT ?a COUNT(DISTINCT ?w) AS ?works
    WHERE{
        ?a wdt:P106 wd:Q42973;
            wdt:P800 ?w.
    }
    GROUP BY ?a
}

} 
GROUP BY ?nationality
ORDER BY DESC(?avgWorks)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('nationality', 'Republic of Florence'), ('avgWorks', '9.2')]
[('nationality', 'Sicyon'), ('avgWorks', '9')]
[('nationality', 'Kingdom of Bulgaria'), ('avgWorks', '8')]
[('nationality', 'Principality of Bulgaria'), ('avgWorks', '8')]
[('nationality', 'First Republic of Austria'), ('avgWorks', '8')]
[('nationality', 'Iraq'), ('avgWorks', '7.666666666666667')]
[('nationality', 'Nazi Germany'), ('avgWorks', '7.5')]
[('nationality', 'Weimar Republic'), ('avgWorks', '6.333333333333333')]
[('nationality', 'Cossack Hetmanate'), ('avgWorks', '6')]
[('nationality', 'Cambodia'), ('avgWorks', '6')]
[('nationality', 'Kingdom of Serbs, Croats and Slovenes'), ('avgWorks', '5.333333333333333')]
[('nationality', 'Principality of Serbia'), ('avgWorks', '5.333333333333333')]
[('nationality', 'Lower Canada'), ('avgWorks', '5')]
[('nationality', 'Kingdom of Serbia'), ('avgWorks', '5')]
[('nationality', 'Wang Jingwei regime'), ('avgWorks', '5')]


15

The result of this query is quite interesting because I retrieved ancient countries that no longer exists. Since this is an average value, such result could be caused by the fact that there are a lot of architects from modern countries with few or no notable works.

## Buildings Properties
This section contains the queries used to answer the second question: _Identify the BGP that connect a building with its architect, type, and architectural style._

As we saw earlier each building type can be inferred by the property "instance of" (wdt:P131), the architect by the property "architect" (wdt:P84) and the architectural style by the property "architectural style" (wdt:P149). Since I did not do it yet I return the architectural style of Pettit Memorial Chapel (wd:Q2081276).

In [15]:
queryString = """
SELECT ?o ?style
WHERE { 

wd:Q2081276 wdt:P149 ?o.
    
?o <http://schema.org/name> ?style.

} 
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q245188'), ('style', 'modern architecture')]


1

Before I move on I check if there are other properties that contains information about the type of building. In order to do it I return all properties connected to any building built by an architect.

In [16]:
queryString = """
SELECT COUNT(DISTINCT ?p) AS ?bProps
WHERE { 

?a wdt:P106 wd:Q42973.
?w wdt:P84 ?a;
    ?p ?o.

} 
"""

print("Results")
run_query(queryString)

Results
[('bProps', '1733')]


1

Since they are a lot I filter them with a regular expression.

In [17]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?a wdt:P106 wd:Q42973.
?w wdt:P84 ?a;
    ?p ?o.
    
?p <http://schema.org/name> ?pname.

FILTER(regex(?pname, ".*type.*") || regex(?pname, ".*building.*")).

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3503'), ('pname', 'LombardiaBeniCulturali building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1305'), ('pname', 'CTBUH Skyscraper Center building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3059'), ('pname', 'Architectuurgids building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P455'), ('pname', 'Emporis building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7630'), ('pname', 'Dictionary of Scottish Architects building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P193'), ('pname', 'main building contractor')]
[('p', 'http://www.wikidata.org/prop/direct/P1699'), ('pname', 'SkyscraperPage building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5208'), ('pname', 'BAG building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5011'), ('pname', 'Prazdne Domy building ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3032'), ('pname', 'adjacent building')]
[('p', 'http://www.wikidata.org/prop/direct/P52

26

I can safely state that the only property containing the type of building is "instance of" (wdt:P31).

### Frank Lloyd Wright Works
Since Frank Lloyd Wright is one of the most famous american architects I count how many different type of building he built throughout his career.

In [18]:
queryString = """
SELECT COUNT(DISTINCT ?type) AS ?buildings
WHERE { 

?b wdt:P84 wd:Q5604;
    wdt:P31 ?type.

} 
"""

print("Results")
run_query(queryString)

Results
[('buildings', '60')]


1

Wright built 60 different type of buildings, but what is the most common type of building he built?

In [19]:
queryString = """
SELECT ?t ?type COUNT(?b) AS ?buildings
WHERE { 

?b wdt:P84 wd:Q5604;
    wdt:P31 ?t.
    
?t <http://schema.org/name> ?type.

} 
GROUP BY ?t ?type
ORDER BY DESC(?buildings)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q1307276'), ('type', 'single-family detached home'), ('buildings', '135')]
[('t', 'http://www.wikidata.org/entity/Q3947'), ('type', 'house'), ('buildings', '20')]
[('t', 'http://www.wikidata.org/entity/Q41176'), ('type', 'building'), ('buildings', '8')]
[('t', 'http://www.wikidata.org/entity/Q5773747'), ('type', 'historic house'), ('buildings', '6')]
[('t', 'http://www.wikidata.org/entity/Q16970'), ('type', 'church building'), ('buildings', '5')]
[('t', 'http://www.wikidata.org/entity/Q811979'), ('type', 'architectural structure'), ('buildings', '5')]
[('t', 'http://www.wikidata.org/entity/Q25037430'), ('type', 'Textile block house'), ('buildings', '4')]
[('t', 'http://www.wikidata.org/entity/Q5783996'), ('type', 'cottage'), ('buildings', '4')]
[('t', 'http://www.wikidata.org/entity/Q1570262'), ('type', 'unfinished building'), ('buildings', '3')]
[('t', 'http://www.wikidata.org/entity/Q1244442'), ('type', 'school building'), ('buildings', 

10

Now I can also return the countries in which he built something. 

In [20]:
queryString = """
SELECT ?country COUNT(?b) AS ?buildings
WHERE { 

?b wdt:P84 wd:Q5604;
    wdt:P17 ?c.
    
?c <http://schema.org/name> ?country.

} 
GROUP BY ?country
"""

print("Results")
run_query(queryString)

Results
[('country', 'United States of America'), ('buildings', '236')]
[('country', 'Canada'), ('buildings', '1')]
[('country', 'Japan'), ('buildings', '3')]


3

Most of his works are in the USA therefore I can see if he worked in a particular city or county by means of the property "located in the administrative territorial entity" wdt:P131.

In [21]:
queryString = """
SELECT ?location COUNT(?b) AS ?buildings
WHERE { 

?b wdt:P84 wd:Q5604;
    wdt:P131 ?l.
    
?l <http://schema.org/name> ?location.

} 
GROUP BY ?location
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('location', 'Oak Park'), ('buildings', '21')]
[('location', 'Chicago'), ('buildings', '11')]
[('location', 'Illinois'), ('buildings', '10')]
[('location', 'Wyoming'), ('buildings', '7')]
[('location', 'Buffalo'), ('buildings', '5')]
[('location', 'Madison'), ('buildings', '5')]
[('location', 'Iowa'), ('buildings', '4')]
[('location', 'New Jersey'), ('buildings', '4')]
[('location', 'Rochester'), ('buildings', '4')]
[('location', 'Okemos'), ('buildings', '4')]
[('location', 'Michigan'), ('buildings', '4')]
[('location', 'New York'), ('buildings', '4')]
[('location', 'Glencoe'), ('buildings', '4')]
[('location', 'Acme'), ('buildings', '3')]
[('location', 'Indiana'), ('buildings', '3')]


15

Most of his works are located in Illinois. In the end, I can also look for the most common architectural style that the buildings built by Wright have.

In [22]:
queryString = """
SELECT ?style COUNT(?b) AS ?buildings
WHERE { 

?b wdt:P84 wd:Q5604;
    wdt:P149 ?s.
    
?s <http://schema.org/name> ?style.

} 
GROUP BY ?style
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('style', 'Prairie School architecture'), ('buildings', '52')]
[('style', 'modern architecture'), ('buildings', '9')]
[('style', 'Queen Anne style architecture'), ('buildings', '5')]
[('style', 'Mayan Revival architecture'), ('buildings', '4')]
[('style', 'Tudor Revival architecture'), ('buildings', '2')]
[('style', 'Chicago School'), ('buildings', '2')]
[('style', 'Shingle style architecture'), ('buildings', '2')]
[('style', 'Usonia'), ('buildings', '2')]
[('style', 'Usonian architecture'), ('buildings', '1')]
[('style', 'Italianate architecture'), ('buildings', '1')]


10

### Other Interesting Queries
I now run some other interesting queries with such information. Firstly I look for the foreign architects that built more buildings in the USA (wd:Q30).

In [23]:
queryString = """
SELECT ?architect ?country COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?b wdt:P84 ?a;
    wdt:P17 wd:Q30.
    
?a wdt:P27 ?c.

?a <http://schema.org/name> ?architect.
?c <http://schema.org/name> ?country.

FILTER(?c != wd:Q30).

} 
GROUP BY ?architect ?country
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('architect', 'Richard Upjohn'), ('country', 'United Kingdom of Great Britain and Ireland'), ('buildings', '48')]
[('architect', 'César Pelli'), ('country', 'Argentina'), ('buildings', '44')]
[('architect', 'Emery Roth'), ('country', 'Hungary'), ('buildings', '44')]
[('architect', 'I. M. Pei'), ('country', 'Taiwan'), ('buildings', '29')]
[('architect', 'Frank Gehry'), ('country', 'Canada'), ('buildings', '22')]
[('architect', 'W. H. Weeks'), ('country', 'Canada'), ('buildings', '20')]
[('architect', 'Paul Philippe Cret'), ('country', 'France'), ('buildings', '18')]
[('architect', 'Eero Saarinen'), ('country', 'Finland'), ('buildings', '18')]
[('architect', 'Rosario Candela'), ('country', 'Kingdom of Italy'), ('buildings', '18')]
[('architect', 'Rosario Candela'), ('country', 'Italy'), ('buildings', '18')]
[('architect', 'Alfred B. Mullett'), ('country', 'United Kingdom of Great Britain and Ireland'), ('buildings', '15')]
[('architect', 'Marcel Breuer'), ('country', 'Kingdom of

15

In the end I look for the most popular architectural styles in the USA. 

In [24]:
queryString = """
SELECT ?style COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?b wdt:P149 ?s;
    wdt:P17 wd:Q30.

?s <http://schema.org/name> ?style.

} 
GROUP BY ?style
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('style', 'Greek Revival architecture'), ('buildings', '2954')]
[('style', 'Renaissance Revival architecture'), ('buildings', '2217')]
[('style', 'Gothic Revival'), ('buildings', '1227')]
[('style', 'Neoclassical architecture'), ('buildings', '1124')]
[('style', 'Queen Anne style architecture'), ('buildings', '1018')]
[('style', 'Colonial Revival architecture'), ('buildings', '808')]
[('style', 'Italianate architecture'), ('buildings', '770')]
[('style', 'National Park Service rustic'), ('buildings', '716')]
[('style', 'Federal architecture'), ('buildings', '549')]
[('style', 'rustic architecture'), ('buildings', '546')]
[('style', 'Georgian architecture'), ('buildings', '528')]
[('style', 'Romanesque Revival architecture'), ('buildings', '506')]
[('style', 'modern architecture'), ('buildings', '454')]
[('style', 'Spanish Colonial Revival architecture'), ('buildings', '451')]
[('style', 'Beaux-Arts'), ('buildings', '427')]


15

## Most Prolific Architects
This section contains the queries used to answer the third question: _Which architect is the most prolific in the U.S. and outside?_

As I said before, in order to assess the most profilic architect I will use the property "architect" (wdt:P84). I do not distinguish between the buildings locations at first. I now return the top-15 most prolific american architects.

In [25]:
queryString = """
SELECT ?a ?architect COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973. 

?b wdt:P84 ?a.

?a <http://schema.org/name> ?architect.

} 
GROUP BY ?architect ?a
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q5604'), ('architect', 'Frank Lloyd Wright'), ('buildings', '244')]
[('a', 'http://www.wikidata.org/entity/Q309483'), ('architect', 'César Pelli'), ('buildings', '87')]
[('a', 'http://www.wikidata.org/entity/Q7411623'), ('architect', 'Samuel Hannaford'), ('buildings', '52')]
[('a', 'http://www.wikidata.org/entity/Q6137444'), ('architect', 'James Knox Taylor'), ('buildings', '51')]
[('a', 'http://www.wikidata.org/entity/Q46868'), ('architect', 'I. M. Pei'), ('buildings', '50')]
[('a', 'http://www.wikidata.org/entity/Q2066635'), ('architect', 'Richard Upjohn'), ('buildings', '49')]
[('a', 'http://www.wikidata.org/entity/Q371506'), ('architect', 'Emery Roth'), ('buildings', '45')]
[('a', 'http://www.wikidata.org/entity/Q706666'), ('architect', 'Albert Kahn'), ('buildings', '45')]
[('a', 'http://www.wikidata.org/entity/Q180374'), ('architect', 'Frank Gehry'), ('buildings', '43')]
[('a', 'http://www.wikidata.org/entity/Q6686579'), ('architect',

15

I discovered Frank Lloyd Wright is the most prolific american architect. Since all the others have fewer building I check that there is no other property that returns an architect's work in both directions.

In [26]:
queryString = """
SELECT ?p ?pname COUNT(DISTINCT ?o) AS ?objs
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973;
    ?p ?o. 

?p <http://schema.org/name> ?pname.

} 
GROUP BY ?p ?pname
ORDER BY DESC(?objs)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('objs', '2582')]
[('p', 'http://www.wikidata.org/prop/direct/P5508'), ('pname', 'archINFORM person or group ID'), ('objs', '2303')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth'), ('objs', '2102')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('pname', 'date of death'), ('objs', '1959')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pname', 'VIAF ID'), ('objs', '1895')]
[('p', 'http://www.wikidata.org/prop/direct/P7859'), ('pname', 'WorldCat Identities ID'), ('objs', '1739')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('pname', 'Library of Congress authority ID'), ('objs', '1449')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('objs', '1367')]
[('p', 'http://www.wikidata.org/prop/direct/P245'), ('pname', 'Union List of Artist Names ID'), ('objs', '1202')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('pnam

25

In [27]:
queryString = """
SELECT ?p ?pname COUNT(DISTINCT ?s) AS ?subjs
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973.

?s ?p ?a.

?p <http://schema.org/name> ?pname.

} 
GROUP BY ?p ?pname
ORDER BY DESC(?subjs)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pname', 'architect'), ('subjs', '7012')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator'), ('subjs', '3196')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant'), ('subjs', '678')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'), ('pname', 'production designer'), ('subjs', '559')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('pname', 'performer'), ('subjs', '252')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pname', 'child'), ('subjs', '234')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author'), ('subjs', '175')]
[('p', 'http://www.wikidata.org/prop/direct/P287'), ('pname', 'designed by'), ('subjs', '172')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer'), ('subjs', '169')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pname', 'father'), ('subjs', '164')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), 

25

I discover there is also the property "creator" (wdt:P170) therefore I return some triples of such property.

In [28]:
queryString = """
SELECT DISTINCT ?s ?sname ?i ?instanceOf
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973.

?s wdt:P170 ?a;
    wdt:P31 ?i.

?s <http://schema.org/name> ?sname.
?i <http://schema.org/name> ?instanceOf.

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q717956'), ('sname', 'Unisphere'), ('i', 'http://www.wikidata.org/entity/Q483453'), ('instanceOf', 'fountain')]
[('s', 'http://www.wikidata.org/entity/Q717956'), ('sname', 'Unisphere'), ('i', 'http://www.wikidata.org/entity/Q860861'), ('instanceOf', 'sculpture')]
[('s', 'http://www.wikidata.org/entity/Q717956'), ('sname', 'Unisphere'), ('i', 'http://www.wikidata.org/entity/Q133792'), ('instanceOf', 'globe')]
[('s', 'http://www.wikidata.org/entity/Q20198613'), ('sname', 'Armchair'), ('i', 'http://www.wikidata.org/entity/Q1847455'), ('instanceOf', 'wing chair')]
[('s', 'http://www.wikidata.org/entity/Q103838652'), ('sname', 'The Mastersingers of Nürnberg'), ('i', 'http://www.wikidata.org/entity/Q3305213'), ('instanceOf', 'painting')]
[('s', 'http://www.wikidata.org/entity/Q103837425'), ('sname', 'Sphynx'), ('i', 'http://www.wikidata.org/entity/Q3305213'), ('instanceOf', 'painting')]
[('s', 'http://www.wikidata.org/entity/Q68140838'), ('sname

25

It looks like this property is not related to buildings. In order to check it I count the number of objects of such property based on what they are instance of.

In [29]:
queryString = """
SELECT ?i ?instanceOf COUNT(DISTINCT ?s) AS ?subjs
WHERE { 

?a wdt:P27 wd:Q30; 
    wdt:P106 wd:Q42973.

?s wdt:P170 ?a;
    wdt:P31 ?i.

?i <http://schema.org/name> ?instanceOf.

} 
GROUP BY ?i ?instanceOf
ORDER BY DESC(?subjs)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('i', 'http://www.wikidata.org/entity/Q3305213'), ('instanceOf', 'painting'), ('subjs', '1267')]
[('i', 'http://www.wikidata.org/entity/Q93184'), ('instanceOf', 'drawing'), ('subjs', '1026')]
[('i', 'http://www.wikidata.org/entity/Q11060274'), ('instanceOf', 'print'), ('subjs', '485')]
[('i', 'http://www.wikidata.org/entity/Q860861'), ('instanceOf', 'sculpture'), ('subjs', '155')]
[('i', 'http://www.wikidata.org/entity/Q838948'), ('instanceOf', 'work of art'), ('subjs', '82')]
[('i', 'http://www.wikidata.org/entity/Q125191'), ('instanceOf', 'photograph'), ('subjs', '35')]
[('i', 'http://www.wikidata.org/entity/Q20437094'), ('instanceOf', 'installation artwork'), ('subjs', '22')]
[('i', 'http://www.wikidata.org/entity/Q20742776'), ('instanceOf', 'video artwork'), ('subjs', '12')]
[('i', 'http://www.wikidata.org/entity/Q179700'), ('instanceOf', 'statue'), ('subjs', '10')]
[('i', 'http://www.wikidata.org/entity/Q4989906'), ('instanceOf', 'monument'), ('subjs', '9')]
[('i', 'http:

25

From this analysis I can assess the only property retrieving an architect's work is "architect" (wdt:P84).

I now return the most prolific architects all over the world.

In [30]:
queryString = """
SELECT ?architect ?country COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P106 wd:Q42973;
    wdt:P27 ?c. 

?b wdt:P84 ?a.

?a <http://schema.org/name> ?architect.
?c <http://schema.org/name> ?country.

} 
GROUP BY ?architect ?country
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('architect', 'Frank Lloyd Wright'), ('country', 'United States of America'), ('buildings', '244')]
[('architect', 'Edwin Lutyens'), ('country', 'United Kingdom'), ('buildings', '172')]
[('architect', 'Reginald Blomfield'), ('country', 'United Kingdom'), ('buildings', '140')]
[('architect', 'Reginald Blomfield'), ('country', 'United Kingdom of Great Britain and Ireland'), ('buildings', '140')]
[('architect', 'Bruno Granholm'), ('country', 'Finland'), ('buildings', '133')]
[('architect', 'Bruno Granholm'), ('country', 'Russian Empire'), ('buildings', '133')]
[('architect', 'Oscar Niemeyer'), ('country', 'Brazil'), ('buildings', '116')]
[('architect', 'Herbert Baker'), ('country', 'United Kingdom'), ('buildings', '109')]
[('architect', 'Herbert Baker'), ('country', 'United Kingdom of Great Britain and Ireland'), ('buildings', '109')]
[('architect', 'Alvar Aalto'), ('country', 'Finland'), ('buildings', '104')]
[('architect', 'Charles Holden'), ('country', 'United Kingdom of Great

15

Now I return the most prolific architects in the US, that is the architects that built most buildings in the USA.

In [31]:
queryString = """
SELECT ?architect ?country COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P106 wd:Q42973;
    wdt:P27 ?c. 

?b wdt:P84 ?a;
    wdt:P17 wd:Q30.

?a <http://schema.org/name> ?architect.
?c <http://schema.org/name> ?country.

} 
GROUP BY ?architect ?country
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('architect', 'Frank Lloyd Wright'), ('country', 'United States of America'), ('buildings', '236')]
[('architect', 'Samuel Hannaford'), ('country', 'United States of America'), ('buildings', '52')]
[('architect', 'James Knox Taylor'), ('country', 'United States of America'), ('buildings', '51')]
[('architect', 'Richard Upjohn'), ('country', 'United Kingdom of Great Britain and Ireland'), ('buildings', '48')]
[('architect', 'Richard Upjohn'), ('country', 'United States of America'), ('buildings', '48')]
[('architect', 'Emery Roth'), ('country', 'Hungary'), ('buildings', '44')]
[('architect', 'César Pelli'), ('country', 'Argentina'), ('buildings', '44')]
[('architect', 'César Pelli'), ('country', 'United States of America'), ('buildings', '44')]
[('architect', 'Emery Roth'), ('country', 'United States of America'), ('buildings', '44')]
[('architect', 'Albert Kahn'), ('country', 'United States of America'), ('buildings', '42')]
[('architect', 'Louis A. Simon'), ('country', 'Unite

15

In the end I return the most prolific american architects in the USA.

In [32]:
queryString = """
SELECT ?a ?architect COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P106 wd:Q42973;
    wdt:P27 wd:Q30. 

?b wdt:P84 ?a;
    wdt:P17 wd:Q30.

?a <http://schema.org/name> ?architect.

} 
GROUP BY ?architect ?a
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q5604'), ('architect', 'Frank Lloyd Wright'), ('buildings', '236')]
[('a', 'http://www.wikidata.org/entity/Q7411623'), ('architect', 'Samuel Hannaford'), ('buildings', '52')]
[('a', 'http://www.wikidata.org/entity/Q6137444'), ('architect', 'James Knox Taylor'), ('buildings', '51')]
[('a', 'http://www.wikidata.org/entity/Q2066635'), ('architect', 'Richard Upjohn'), ('buildings', '48')]
[('a', 'http://www.wikidata.org/entity/Q371506'), ('architect', 'Emery Roth'), ('buildings', '44')]
[('a', 'http://www.wikidata.org/entity/Q309483'), ('architect', 'César Pelli'), ('buildings', '44')]
[('a', 'http://www.wikidata.org/entity/Q706666'), ('architect', 'Albert Kahn'), ('buildings', '42')]
[('a', 'http://www.wikidata.org/entity/Q6686579'), ('architect', 'Louis A. Simon'), ('buildings', '42')]
[('a', 'http://www.wikidata.org/entity/Q7287147'), ('architect', 'Ralph Adams Cram'), ('buildings', '36')]
[('a', 'http://www.wikidata.org/entity/Q464603'), (

15

## Location of Buildings Designed by American Architects
This section contains the queries used to answer the last question: _Analyze the location of the buildings designed by the american architects answering the following questions:_
 
   _4.1 Which styles exists in the U.S.? Which type of building?_
   
   _4.2 In which U.S. states there are more notable buildings?_
   
   _4.3 Which U.S. state contains the largest number of building designed by U.S. architects_

### Styles and Type of Building
I now answer to the first question about styles and type of buildings in the USA. 

Firstly I return how many different styles there are in the USA (wd:Q30). I recall each building contains information about the architectural style by means of the property "architectural style" (wdt:P149).

In [33]:
queryString = """
SELECT COUNT(DISTINCT ?s) AS ?styles
WHERE { 

?b wdt:P17 wd:Q30;
    wdt:P149 ?s.

} 
"""

print("Results")
run_query(queryString)

Results
[('styles', '222')]


1

Since they are a lot I return only the most popular ones.

In [34]:
queryString = """
SELECT ?s ?style COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?b wdt:P17 wd:Q30;
    wdt:P149 ?s.
    
?s <http://schema.org/name> ?style.

} 
GROUP BY ?s ?style
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1513688'), ('style', 'Greek Revival architecture'), ('buildings', '2954')]
[('s', 'http://www.wikidata.org/entity/Q502163'), ('style', 'Renaissance Revival architecture'), ('buildings', '2217')]
[('s', 'http://www.wikidata.org/entity/Q186363'), ('style', 'Gothic Revival'), ('buildings', '1227')]
[('s', 'http://www.wikidata.org/entity/Q54111'), ('style', 'Neoclassical architecture'), ('buildings', '1124')]
[('s', 'http://www.wikidata.org/entity/Q529819'), ('style', 'Queen Anne style architecture'), ('buildings', '1018')]
[('s', 'http://www.wikidata.org/entity/Q5148367'), ('style', 'Colonial Revival architecture'), ('buildings', '808')]
[('s', 'http://www.wikidata.org/entity/Q615196'), ('style', 'Italianate architecture'), ('buildings', '770')]
[('s', 'http://www.wikidata.org/entity/Q6974748'), ('style', 'National Park Service rustic'), ('buildings', '716')]
[('s', 'http://www.wikidata.org/entity/Q1400086'), ('style', 'Federal architecture'

15

I return the cities or counties where there are more Greek Revival architecture (wd:Q1513688) style buidlings.

In [35]:
queryString = """
SELECT ?l ?location COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?b wdt:P17 wd:Q30;
    wdt:P149 wd:Q1513688;
    wdt:P131 ?l.
    
?l <http://schema.org/name> ?location.

} 
GROUP BY ?l ?location
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('l', 'http://www.wikidata.org/entity/Q1384'), ('location', 'New York'), ('buildings', '335')]
[('l', 'http://www.wikidata.org/entity/Q1370'), ('location', 'Virginia'), ('buildings', '197')]
[('l', 'http://www.wikidata.org/entity/Q173'), ('location', 'Alabama'), ('buildings', '127')]
[('l', 'http://www.wikidata.org/entity/Q1371'), ('location', 'West Virginia'), ('buildings', '114')]
[('l', 'http://www.wikidata.org/entity/Q1400'), ('location', 'Pennsylvania'), ('buildings', '102')]
[('l', 'http://www.wikidata.org/entity/Q1397'), ('location', 'Ohio'), ('buildings', '94')]
[('l', 'http://www.wikidata.org/entity/Q771'), ('location', 'Massachusetts'), ('buildings', '90')]
[('l', 'http://www.wikidata.org/entity/Q1588'), ('location', 'Louisiana'), ('buildings', '70')]
[('l', 'http://www.wikidata.org/entity/Q1393'), ('location', 'Delaware'), ('buildings', '57')]
[('l', 'http://www.wikidata.org/entity/Q779'), ('location', 'Connecticut'), ('buildings', '54')]
[('l', 'http://www.wikidata

15

Now I return how many different type of buildings there are in the USA. I recall the information about the type of buildings is inferred by means of the property "instance of" (wdt:P31).

In [36]:
queryString = """
SELECT ?t ?type COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P106 wd:Q42973.

?b wdt:P17 wd:Q30;
    wdt:P84 ?a;
    wdt:P31 ?t.
    
?t <http://schema.org/name> ?type.

} 
GROUP BY ?t ?type
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q1307276'), ('type', 'single-family detached home'), ('buildings', '771')]
[('t', 'http://www.wikidata.org/entity/Q16970'), ('type', 'church building'), ('buildings', '731')]
[('t', 'http://www.wikidata.org/entity/Q41176'), ('type', 'building'), ('buildings', '580')]
[('t', 'http://www.wikidata.org/entity/Q11303'), ('type', 'skyscraper'), ('buildings', '357')]
[('t', 'http://www.wikidata.org/entity/Q64037785'), ('type', 'county courthouse'), ('buildings', '300')]
[('t', 'http://www.wikidata.org/entity/Q27686'), ('type', 'hotel'), ('buildings', '295')]
[('t', 'http://www.wikidata.org/entity/Q3947'), ('type', 'house'), ('buildings', '259')]
[('t', 'http://www.wikidata.org/entity/Q24354'), ('type', 'theatre'), ('buildings', '243')]
[('t', 'http://www.wikidata.org/entity/Q41253'), ('type', 'movie theater'), ('buildings', '218')]
[('t', 'http://www.wikidata.org/entity/Q13402009'), ('type', 'apartment building'), ('buildings', '191')]
[('t', 'ht

15

As expected the most common building type is "single-family detached home" that are regular houses.

### US States with More Notable Buildings

I now return the US states with more notable buildings. In order to do it I use the property "notable work" (wdt:P800) to return all the notable works and then I use the property "located in the administrative territorial entity" (wdt:P131) to inferred the location.

In [37]:
queryString = """
SELECT ?l ?location COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P106 wd:Q42973;
    wdt:P800 ?b.

?b wdt:P17 wd:Q30;
    wdt:P131 ?l.
    
?l <http://schema.org/name> ?location.

} 
GROUP BY ?l ?location
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('l', 'http://www.wikidata.org/entity/Q11299'), ('location', 'Manhattan'), ('buildings', '74')]
[('l', 'http://www.wikidata.org/entity/Q61'), ('location', 'Washington, D.C.'), ('buildings', '44')]
[('l', 'http://www.wikidata.org/entity/Q1297'), ('location', 'Chicago'), ('buildings', '31')]
[('l', 'http://www.wikidata.org/entity/Q99'), ('location', 'California'), ('buildings', '24')]
[('l', 'http://www.wikidata.org/entity/Q65'), ('location', 'Los Angeles'), ('buildings', '21')]
[('l', 'http://www.wikidata.org/entity/Q771'), ('location', 'Massachusetts'), ('buildings', '15')]
[('l', 'http://www.wikidata.org/entity/Q1400'), ('location', 'Pennsylvania'), ('buildings', '12')]
[('l', 'http://www.wikidata.org/entity/Q1204'), ('location', 'Illinois'), ('buildings', '11')]
[('l', 'http://www.wikidata.org/entity/Q812'), ('location', 'Florida'), ('buildings', '10')]
[('l', 'http://www.wikidata.org/entity/Q779'), ('location', 'Connecticut'), ('buildings', '10')]
[('l', 'http://www.wikidat

15

### US States with more buildings designed by US architect

In the end I return the US states with the highest number of buildings designed by US architects. In this case I use the property "architect" (wdt:P84) because I am interested in all architects' works, not only the notable works.

In [38]:
queryString = """
SELECT ?l ?location COUNT(DISTINCT ?b) AS ?buildings
WHERE { 

?a wdt:P106 wd:Q42973;
    wdt:P27 wd:Q30.

?b wdt:P84 ?a;
    wdt:P17 wd:Q30;
    wdt:P131 ?l.
    
?l <http://schema.org/name> ?location.

} 
GROUP BY ?l ?location
ORDER BY DESC(?buildings)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('l', 'http://www.wikidata.org/entity/Q11299'), ('location', 'Manhattan'), ('buildings', '599')]
[('l', 'http://www.wikidata.org/entity/Q61'), ('location', 'Washington, D.C.'), ('buildings', '202')]
[('l', 'http://www.wikidata.org/entity/Q1384'), ('location', 'New York'), ('buildings', '189')]
[('l', 'http://www.wikidata.org/entity/Q771'), ('location', 'Massachusetts'), ('buildings', '159')]
[('l', 'http://www.wikidata.org/entity/Q1297'), ('location', 'Chicago'), ('buildings', '158')]
[('l', 'http://www.wikidata.org/entity/Q1546'), ('location', 'Iowa'), ('buildings', '127')]
[('l', 'http://www.wikidata.org/entity/Q1439'), ('location', 'Texas'), ('buildings', '119')]
[('l', 'http://www.wikidata.org/entity/Q99'), ('location', 'California'), ('buildings', '119')]
[('l', 'http://www.wikidata.org/entity/Q1397'), ('location', 'Ohio'), ('buildings', '115')]
[('l', 'http://www.wikidata.org/entity/Q1400'), ('location', 'Pennsylvania'), ('buildings', '112')]
[('l', 'http://www.wikidata.

15

Again, Manhattan has the biggest number of buildings.